### import library

In [1]:
import requests
import datetime
import re
import pandas as pd 
import numpy as np
from requests_html import HTML
from requests_html import HTMLSession
from bs4 import BeautifulSoup

### Crawl link from nhadat24h.vn

In [2]:
arr = []
try:
    for page in range(38):
        page= str(page+1)
        link ="https://nhadat24h.net/ban-dat-nen-du-an-da-nang/page"+page
        arr.append(link)
    df = pd.DataFrame(arr,columns=['links'])
except KeyboardInterrupt:
    pass  # Xử lý ngoại lệ khi người dùng gián đoạn chương trình
finally:
    df = pd.DataFrame(arr, columns=['links'])


### Export file csv

In [3]:
df.to_csv('nhadat24h.csv', index=True)
print("hoàn thành")

hoàn thành


### View a head of file nhadat24h.csv

In [4]:
frame = pd.read_csv('nhadat24h.csv')
frame.head()

,Unnamed: 0,links
0,0,https://nhadat24h.net/ban-dat-nen-du-an-da-nan...
1,1,https://nhadat24h.net/ban-dat-nen-du-an-da-nan...
2,2,https://nhadat24h.net/ban-dat-nen-du-an-da-nan...
3,3,https://nhadat24h.net/ban-dat-nen-du-an-da-nan...
4,4,https://nhadat24h.net/ban-dat-nen-du-an-da-nan...


### Crawl data from each link

In [8]:
Price, Area = [],[]
Address, Way_in, Width, Direction =[], [], [], []
Land_type = []


In [9]:
def getdata(link):
    session = HTMLSession()
    response = session.get(link)
    # soup = BeautifulSoup(response.text,'html.parser')

    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        #lấy tất cả các thẻ a có trong 1 page
        a_tags = soup.find_all('div', class_='dv-item')

        for a_tag in a_tags:

            #Price
            price_tag = a_tag.find('label', class_='a-txt-cl1')
            if price_tag:
                price = price_tag.text.strip()
            else:
                price = "NaN"   
            Price.append(price)


            #Area
            area_tag = a_tag.find('label', class_='a-txt-cl2')
            if area_tag:
                area = area_tag.text.strip()
            else:
                area = "NaN"
            Area.append(area)


            #Address
            address_tag = a_tag.find('label', class_='rvVitri')
            if address_tag:
                address = address_tag.text.strip()
            else:
                address ="NaN"
            Address.append(address)


            #Way_in
            way_in_tags = a_tag.find_all(lambda tag: tag.name == 'span' and tag.find('i', class_='fas fa-road'))
            way_ins = []
            if way_in_tags:
                for way_in_tag in way_in_tags:
                    way_in_text = way_in_tag.text.strip()
                    if "Đường vào" in way_in_text:
                        way_ins.append(way_in_text)
            way_in = ', '.join(way_ins) if way_ins else "NaN"
            Way_in.append(way_in)


            #Width
            width_tags = a_tag.find_all(lambda tag: tag.name == 'span' and tag.find('i', class_='fa-arrows-alt'))
            widths = []
            if width_tags:
                for width_tag in width_tags:
                    width_text = width_tag.text.strip()
                    if "Mặt tiền" in width_text:
                        widths.append(width_text)
            width = ', '.join(widths) if widths else "NaN"
            Width.append(width)

    
            #Direction
            direction_tags = a_tag.find_all(lambda tag: tag.name == 'span' and tag.find('i', class_='fas fa-compass'))
            directions = []
            if direction_tags:
                for direction_tag in direction_tags:
                    direction = direction_tag.text.strip()
                    directions.append(direction)
                direction = ', '.join(directions)
            else:
                direction = "NaN"
            Direction.append(direction)


            #Land_type
            div_land_type_tag = soup.find('div', class_='review1')
            if div_land_type_tag:
                land_type_tags = div_land_type_tag.find_all('span', class_='ex3')
                if land_type_tags:
                    land_type = land_type_tags[0].text.strip()
                else:
                    land_type = "NaN"
            else:
                land_type = "NaN"
            Land_type.append(land_type)

    df = pd.DataFrame({ 
                        'Gia': Price, 'Dien tich': Area, 
                        'Dia chi': Address,  
                        'Duong vao':Way_in, 
                        'Chieu ngang': Width,
                        'Huong nha':Direction, 'Loai hinh dat' :Land_type
                    })
    df = pd.DataFrame(df)
    df.to_csv('data_crawl_nhadat24h.csv', encoding="utf-8-sig", index=False)

# Thực hiện trích xuất dữ liệu từ mỗi liên kết trong dataframe
for link in frame['links']:
    data = getdata(link)